## 1. Collecting relevant data

In this step, we collected the quantitative data that could be used by a ML system to make predictions. Before collecting this data, we needed to identify the channels through which an ML system could realistically harvest data within the identified domestic practice. These channels could be, for instance: (1) device sensors that track user interactions; (2) wearable sensors that monitor biometric data, the user’s current activity, or their location; (3) fixed sensors that detect environmental data, such as temperature, occupancy, or light intensity; (4) services that collect information about their users' behavior, such as purchase patterns or resource consumption patterns. The data that was collected for the grocery automation case study was in the form of grocery receipts(figure~\ref{fig:receipts}). This data is automatically collected by supermarkets, and can be linked to individual customers through their customer cards or online purchases.

In this notebook we:
1. Import the data we collected in an excel file
2. Preview our dataset

### Import libraries 

In [1]:
#Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
import dataframe_image as dfi
from matplotlib.ticker import StrMethodFormatter

### Load and view data 

In [2]:
# The data has been collected in an excel file, and needs to be converted to csv:
read_file = pd.read_excel (open (r"/workspaces/Plenty-in-the-Pantry/database/Groceries_onehousehold.xlsx", 'rb'), sheet_name='Household 1m+1f rural')
read_file.to_csv (r"/workspaces/Plenty-in-the-Pantry/database/Groceries_onehousehold.csv", index = None, header=True)

In [3]:
df = pd.read_csv (r"/workspaces/Plenty-in-the-Pantry/database/Groceries_onehousehold.csv")

del df["HH"]
df.describe(include='all')

,week,order_ID,item_id,amount,price_unit,price_total,date,day,timestamp,time,store_type,store_name,promo,item_type,category
count,372.000000,372.000000,372,372.000000,372.000000,372.000000,372,372,372,372,372,372,372.000000,372,372
unique,NaN,NaN,314,NaN,NaN,NaN,26,7,35,4,5,9,NaN,126,16
top,NaN,NaN,GROF BROOD GESN.,NaN,NaN,NaN,2022-01-08,Saturday,17:25:00,morning,supermarket,Okay,NaN,charcuterie,fruit & vegetables
freq,NaN,NaN,6,NaN,NaN,NaN,45,87,44,186,306,127,NaN,25,103
mean,4.056452,18.881720,NaN,1.274194,2.602328,2.867247,NaN,NaN,NaN,NaN,NaN,NaN,0.083333,NaN,NaN
std,2.067470,10.908193,NaN,1.103819,1.947725,2.019200,NaN,NaN,NaN,NaN,NaN,NaN,0.276758,NaN,NaN
min,1.000000,1.000000,NaN,1.000000,0.064680,0.064680,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
25%,2.000000,8.000000,NaN,1.000000,1.300000,1.543970,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
50%,4.000000,20.000000,NaN,1.000000,2.240250,2.462860,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
75%,5.000000,26.250000,NaN,1.000000,3.290000,3.875000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN


In [4]:
df.head()

,week,order_ID,item_id,amount,price_unit,price_total,date,day,timestamp,time,store_type,store_name,promo,item_type,category
0,1,5,RABEKO choco light 250g,2,2.82,5.64,2021-11-23,Tuesday,12:32:00,noon,supermarket,Okay,0,chocolate spread,breakfast & spreads
1,1,5,JOYVALLE pudding griesmeel natuur 135g,4,0.99,3.96,2021-11-23,Tuesday,12:32:00,noon,supermarket,Okay,0,pudding,dairy & plant based
2,1,5,BONI tomatensoep met balletjes 950ml,1,1.99,1.99,2021-11-23,Tuesday,12:32:00,noon,supermarket,Okay,0,soup,canned foods
3,1,5,LIEBIG DELISOUP 9 groenten brik 1L,1,2.59,2.59,2021-11-23,Tuesday,12:32:00,noon,supermarket,Okay,0,soup,canned foods
4,1,5,LIEBIG DELISOUP tom. Balletjes brik 1L,1,2.59,2.59,2021-11-23,Tuesday,12:32:00,noon,supermarket,Okay,0,soup,canned foods


In [5]:
# new row for reordered items 
df['reorder'] = df.item_type.duplicated()
# convert reorder row to boolean value
df['reorder'] = df['reorder'].astype(bool)
#create an ID for each item 
df.rename(columns={'item_id':'item_name'}, inplace=True)
df['item_id'] = pd.factorize(df['item_name'])[0]
#create an ID for each type 
df['type_id'] = pd.factorize(df['item_type'])[0]
#first make a new column for the amount of items per order
df['order_amount'] = df.groupby('order_ID')['amount'].transform('sum')
# make a new column for the price per order
df['order_price'] = df.groupby('order_ID')['price_total'].transform('sum')

In [6]:
df['week'] = df['week'].astype(int)

df['order_ID'] = df['order_ID'].astype(int)

df['amount'] = df['amount'].astype(int)

df['promo'] = df['promo'].astype(bool)

df['item_id'] = df['item_id'].astype(int)

df['type_id'] = df['type_id'].astype(int)

In [7]:
df_des = df.describe(include='all', datetime_is_numeric=True)

df_styled = df_des.style.background_gradient() #adding a gradient based on values in cell
dfi.export(df_styled,"describe.png")

TypeError: NDFrame.describe() got an unexpected keyword argument 'datetime_is_numeric'

In [67]:
df.dtypes

date_timestamp    datetime64[ns]
week                       int32
order_ID                   int32
item_name                 object
amount                     int32
price_unit               float64
price_total              float64
day                       object
time                      object
store_type                object
store_name                object
promo                       bool
item_type                 object
category                  object
reorder                     bool
item_id                    int32
type_id                    int32
order_amount               int64
order_price              float64
dtype: object

### New (numeric) variables for the confusion matrix

In [68]:
# Recode dow

df['day_num']=df['day'].astype('category').cat.codes
# Recode store_type

df['storetype_num']=df['store_type'].astype('category').cat.codes
# Recode store_name

df['storename_num']=df['store_name'].astype('category').cat.codes
# Recode categories

df['cat_num']=df['category'].astype('category').cat.codes
# Recode time

df['time_num']=df['time'].astype('category').cat.codes
# Recode time

df['promo_num']=df['promo'].astype('category').cat.codes
df.dtypes

#### convert time and date columns

In [75]:
df['datetime'] = pd.to_datetime(df.date_timestamp)
del df["date_timestamp"]

In [76]:
df['timestamp']=pd.to_numeric(df['datetime'])

In [77]:
df['dates'] = pd.to_datetime(df['datetime']).dt.date
df['times'] = pd.to_datetime(df['datetime']).dt.time

In [78]:
df['times_min'] = df.datetime.dt.hour *100 + df.datetime.dt.minute
# df['dates_hours']=df['dates'].astype('category').cat.codes

df['dates_days']=df.datetime.dt.year * 10000 + df.datetime.dt.month * 100 + df.datetime.dt.day 

In [79]:
# df[['datetime', 'timestamp', 'dates', 'times', 'dates_num', 'times_num']].head()
df[['datetime', 'timestamp', 'dates', 'times', 'times_min', 'dates_days']].head()

,datetime,timestamp,dates,times,times_min,dates_days
0,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
1,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
2,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
3,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
4,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123


In [80]:
df.dtypes

week                      int32
order_ID                  int32
item_name                object
amount                    int32
price_unit              float64
price_total             float64
day                      object
time                     object
store_type               object
store_name               object
promo                      bool
item_type                object
category                 object
reorder                    bool
item_id                   int32
type_id                   int32
order_amount              int64
order_price             float64
day_num                    int8
storetype_num              int8
storename_num              int8
cat_num                    int8
time_num                   int8
promo_num                  int8
datetime         datetime64[ns]
timestamp                 int64
dates                    object
times                    object
times_min                 int64
dates_days                int64
dtype: object

## Save the final dataframe as a new csv file

In [81]:
df.head()

,week,order_ID,item_name,amount,price_unit,price_total,day,time,store_type,store_name,...,storename_num,cat_num,time_num,promo_num,datetime,timestamp,dates,times,times_min,dates_days
0,1,5,RABEKO choco light 250g,2,2.82,5.64,Tuesday,noon,supermarket,Okay,...,6,2,3,0,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
1,1,5,JOYVALLE pudding griesmeel natuur 135g,4,0.99,3.96,Tuesday,noon,supermarket,Okay,...,6,7,3,0,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
2,1,5,BONI tomatensoep met balletjes 950ml,1,1.99,1.99,Tuesday,noon,supermarket,Okay,...,6,3,3,0,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
3,1,5,LIEBIG DELISOUP 9 groenten brik 1L,1,2.59,2.59,Tuesday,noon,supermarket,Okay,...,6,3,3,0,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123
4,1,5,LIEBIG DELISOUP tom. Balletjes brik 1L,1,2.59,2.59,Tuesday,noon,supermarket,Okay,...,6,3,3,0,2021-11-23 12:32:00,1637670720000000000,2021-11-23,12:32:00,1232,20211123


In [82]:
df.to_csv(r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv", index=False)
df.drop(["storename_num", 'cat_num', 'time_num', 'promo_num', 'timestamp', 'dates', 'times', 'dates_days'], axis=1, inplace=True)
df.drop(['day_num', 'storetype_num', 'reorder', 'times_min'], axis=1, inplace=True)
df.head()